## Data compilation

In [1]:
import pandas as pd
import numpy as np

## Combining dataset

We combine together training and test dataset, so that we can use the same variables and imputation methods across both dataset. We assign home price (SalePrice) values for test dataset at $99. This way we can separate datasets on next steps.

In [5]:
train = pd.read_csv("./data/train.csv")
train.shape

(1460, 81)

In [6]:
test = pd.read_csv("./data/test.csv")
test.shape

(1459, 80)

In [9]:
# Add SalePrice = 99 to test set
test['SalePrice'] = 99
test.shape

(1459, 81)

In [12]:
# Get a combined dataset
dataset = test.append(train)
dataset.shape

(2919, 81)

We also create a variable Month that is equal to year*100+month. It will help us to join time series later

In [14]:
dataset['Month'] = dataset['YrSold'] * 100 + dataset['MoSold']
dataset.Month.head()

0    201006
1    201006
2    201003
3    201006
4    201001
Name: Month, dtype: int64

In [15]:
# Save the dataset
dataset.to_csv("./data/dataset.csv", index=False)

## Merging with time-series datasets

We are adding the following variables that may explain dynamics of housing prices:

* Dow Jones Real Estate index (lagged by one month)

* Fannie Mae mortgage rates (lagged)

* Labor force and Unemployment rate in Ames (lagged)

In [50]:
dataset = pd.read_csv('./data/dataset.csv')

In [51]:
# A helper function that creates Month variable
def createMonth(date):
    '''
    A helper function that takes a date and returns an integer YYYYMM
    '''
    year = int(date.strftime('%Y'))
    month = int(date.strftime('%m'))
    
    return (year*100 +month)

# Dow Jones Real estate index
djre = pd.read_csv("./data/djre.csv", parse_dates=True)

djre['Date'] = pd.to_datetime(djre['Date']) #Convert to datetime

djre['Month'] = djre['Date'].apply(lambda x: createMonth(x))
djre = djre.groupby('Month').last().reset_index() # Keep end of month observation
djre.set_index('Date', inplace=True) # Set date index to use shift method later

djre = djre.shift(1) #Create a lagged variable

# fannie mae mortgage rates 
fannie = pd.read_csv('./data/FannieMaeRates.csv', parse_dates=True)

fannie['Date'] = pd.to_datetime(fannie['Date']) #Convert to datetime

fannie['Month'] = fannie['Date'].apply(lambda x: createMonth(x))
fannie = fannie.groupby('Month').last().reset_index() # Keep end of month observation
fannie.set_index('Date', inplace=True) # Set date index to use shift method later

fannie = fannie.shift(1) #Create a lagged variable
fannie.columns = ['Month', 'Fannie']


# Ames unemployment rate
unempl = pd.read_csv("./data/unempl.csv", parse_dates=True)

unempl['Month'] = unempl['Year']*100 + unempl['Month']
unempl['Date'] = pd.to_datetime(unempl['Month'], format='%Y%m') # Still need date to create lagged
unempl.drop(['Year', 'employment', 'unemployment'], axis=1, inplace=True)

unempl.set_index('Date', inplace=True) # Set date index to use shift method later

unempl = unempl.shift(1) #Create a lagged variable
unempl.columns = ['Month', 'LaborForce', 'UnemplRate']


# Combine with the dataset
dataset = pd.merge(dataset, djre, on='Month', how='left')
dataset = pd.merge(dataset, fannie, on='Month', how='left')
dataset = pd.merge(dataset, unempl, on='Month', how='left')





In [52]:
dataset.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Month,DJREI,Fannie,LaborForce,UnemplRate
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,6,2010,WD,Normal,99,201006,183.00,4.561,51950.0,4.7
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,6,2010,WD,Normal,99,201006,183.00,4.561,51950.0,4.7
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,3,2010,WD,Normal,99,201003,193.37,4.762,53639.0,4.6
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,6,2010,WD,Normal,99,201006,183.00,4.561,51950.0,4.7
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,1,2010,WD,Normal,99,201001,168.83,4.993,52631.0,4.8


## Imputation of missingness

In [56]:
missing_data = dataset.isna().sum(axis=0)
missing_data[missing_data >0 ]

MSZoning           4
LotFrontage      486
Alley           2721
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        24
MasVnrArea        23
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinSF1         1
BsmtFinType2      80
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
Electrical         1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu     1420
GarageType       157
GarageYrBlt      159
GarageFinish     159
GarageCars         1
GarageArea         1
GarageQual       159
GarageCond       159
PoolQC          2909
Fence           2348
MiscFeature     2814
SaleType           1
dtype: int64

In [57]:
missing_data/dataset.shape[0]  # As percentage of data

Id               0.000000
MSSubClass       0.000000
MSZoning         0.001370
LotFrontage      0.166495
LotArea          0.000000
Street           0.000000
Alley            0.932169
LotShape         0.000000
LandContour      0.000000
Utilities        0.000685
LotConfig        0.000000
LandSlope        0.000000
Neighborhood     0.000000
Condition1       0.000000
Condition2       0.000000
BldgType         0.000000
HouseStyle       0.000000
OverallQual      0.000000
OverallCond      0.000000
YearBuilt        0.000000
YearRemodAdd     0.000000
RoofStyle        0.000000
RoofMatl         0.000000
Exterior1st      0.000343
Exterior2nd      0.000343
MasVnrType       0.008222
MasVnrArea       0.007879
ExterQual        0.000000
ExterCond        0.000000
Foundation       0.000000
                   ...   
Fireplaces       0.000000
FireplaceQu      0.486468
GarageType       0.053786
GarageYrBlt      0.054471
GarageFinish     0.054471
GarageCars       0.000343
GarageArea       0.000343
GarageQual  